In [1]:
import bitboards
import autoencoder
import pgn_reader as reader
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import gc


gc.collect()
torch.cuda.empty_cache()

dsetSize, batch = 2**12, 2**8
#autoEncoder = autoencoder.autoencoder().cuda()
autoEncoder =  torch.load('autoencoder10.pt').cuda()
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(autoEncoder.parameters())

class chessDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data = data_list
        
    def __getitem__(self, idx):
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)


data = reader.get_dataset(dsetSize)
while data:
    random.shuffle(data)
    
    dataset = chessDataset(data[:math.floor(1/16*len(data))])
    testset = chessDataset(data[math.floor(1/16*len(data)):-1])

    DataLoader = torch.utils.data.DataLoader(dataset, batch_size = batch, shuffle = True)
    TestLoader = torch.utils.data.DataLoader(testset, batch_size = batch, shuffle = True)   
    
    autoEncoder.eval()
    test_loss = 0
    for x in TestLoader:
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        test_loss += loss.item()
    print("Test loss: ", test_loss / len(testset))
        
    autoEncoder.train()
    training_loss = 0
    for x in DataLoader:
        optimizer.zero_grad()
        x = x.cuda()
        y = autoEncoder(x).cuda()
        loss = criterion(y, x)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    print("Training loss: ", training_loss / len(dataset), end='\n\n')
    
    data = reader.get_dataset(dsetSize)
    
    torch.save(autoEncoder, 'autoencoder11.pt')

Test loss:  23.499946936846452
Training loss:  23.268906817933306

Test loss:  52.20996283594965
Training loss:  52.52725535144138

Test loss:  35.56426353553673
Training loss:  35.83979797363281

Test loss:  32.480644626786045
Training loss:  32.21307373046875

Test loss:  29.443502197265627
Training loss:  30.158428192138672

Test loss:  31.043002153104524
Training loss:  32.186076985362874

Test loss:  29.59578747800133
Training loss:  28.891141818679927

Test loss:  29.817503152395236
Training loss:  29.980463853522913

Test loss:  31.49512505293563
Training loss:  31.181883767934945

Test loss:  29.96443467987496
Training loss:  30.28660792784932

Test loss:  27.342071586595537
Training loss:  27.61166794855792

Test loss:  25.37476400351925
Training loss:  24.503906422741007

Test loss:  26.196085529443458
Training loss:  26.212962725690304

Test loss:  25.81423464688388
Training loss:  24.88454747754474

Test loss:  23.960987909602732
Training loss:  24.017158479291677

Test los

KeyboardInterrupt: 